В тетрадке собран код для проверки управления и снятия омонимии существительных. Будут добавлены примеры

In [ ]:
import conllu

In [2]:
def read_conllu(conllu_file):
    with open(conllu_file, "r", encoding="utf-8") as f:
        data = f.read()
        return conllu.parse(data)

In [3]:
pet_depression_sents = read_conllu('text_to_test_control_cases.conllu')

In [4]:
def get_adjectives_dict(sentence): ##выделяет прилагательные, которые затем будут проверяться
    adjectives_dict = {'full': [], 'predicative': []}
    adjectives = sentence.filter(upostag='ADJ')
    for adjective in adjectives:
        if 'Case' in adjective['feats']:
            adjectives_dict['full'].append(adjective)
        else:
            adjectives_dict['predicative'].append(adjective)

In [5]:
def get_number_or_gender(word):
    try: 
        if word['feats']['Number'] == 'Plur':
            agreement_feature = 'Plur'
        else:
            agreement_feature = word['feats']['Gender']
    except KeyError: ## если по какой-то причине нет согласовательной категории в словаре
        agreement_feature = None
    return agreement_feature

In [ ]:
def get_declenation(lemma, gender, animacy): ##склонение нужно для дальнейшего снятия омонимии
    if animacy = 'Inan':
        postfix = '_inanim'
    else:
        postfix = '_anim'
    last_letter = noun_lemma[-1]
    decl_n = None
    if last_letter in 'ая':
        if lemma.endwith('ия'):
            dec_n = '1ия'
        else:
            dec_n = '1'
    elif last_letter in 'ое':
        decl_n = '2_neutro'
    elif last_letter in 'йцкнгшщзхфвпрлджчсмтб':
        decl_n is '2_musc'
    elif last_letter == 'ь':
        if noun_lemma[-2] in 'чжшщ' or gender == 'Fem':
            decl_n = '3'
        else:
            decl_n = '2_masc'
    if decl_n:
        return decl_n + postfix

In [ ]:
##Информацию о именительном и родительном мужского рода второго склонения нужно добавлять словарно

homonimic_dict = {'1_anim': [[('Nom', 'Plur'), ('Gen', 'Sing')], [('Dat', 'Sing'), ('Loc', 'Sing')], [('Gen', 'Pl'), ('Acc', 'Pl')]],
                '1ия_anim': [[('Nom', 'Plur'), ('Gen', 'Sing'), ('Dat', 'Sing')], [('Gen', 'Pl'), ('Acc', 'Pl')]],
                '1_inanim': [[('Nom', 'Plur'), ('Acc', 'Plur'), ('Gen', 'Sing')], [('Dat', 'Sing'), ('Loc', 'Sing')]]
                '1ия_anim': [[('Nom', 'Plur'), ('Acc', 'Plur'), ('Gen', 'Sing'), ('Dat', 'Sing')]],
                '2_neutro_inanim': [[('Nom', 'Sing'), ('Acc', 'Sing')], [('Gen', 'Sing'), ('Nom', 'Plur'), ('Acc', 'Plur')]],
                '2_masc_inanim': [[('Nom', 'Sing'), ('Acc', 'Sing')]]
                '2_anim': [[('Nom', 'Sing'), ('Acc', 'Sing')], [('Gen', 'Pl'), ('Acc', 'Plur')]],
                '3_inanim': [[('Nom', 'Sing'), ('Acc', 'Sing')], [('Gen', 'Sing'), ('Dat', 'Sing'), ('Loc', 'Sing'), ('Nom', 'Pl'), ('Acc', 'Pl')]],
                '3_anim':  [[('Nom', 'Sing'), ('Acc', 'Sing')], [('Gen', 'Sing'), ('Dat', 'Sing'), ('Loc', 'Sing'), ('Nom', 'Pl')], [('Gen', 'Pl'), ('Acc', 'Pl')]]


In [ ]:
##Функция принимает существительное и возвращает его возможные тэгсеты (чтобы не выявлять ложные ошибки из-за омонимии)
def get_possible_tags_noun(udpipe_noun, homonimic_dict):
    declention = get_declenation(udpipe_noun['lemma'], udpipe_noun['feats']['Gender'], udpipe_noun['feats']['Animacy'])
    agreement_tags = (udpipe_noun['feats']['Case'], udpipe_noun['feats']['Number'])
    if declention:
        homonimic_lists = homonimic_dict[declention]
    options = [agreement_tags]
    for homonimic_list in homonimic_lists:
        if agreement_tags in homonimic_list:
            options = homonimic_list
            break
    return options

In [ ]:
##Проверяет, может ли существительное быть согласовано с указанным падежом и родом/числом
def check_agreement_homonimy(noun, case, gender_or_number): ##проверка омонимии для согласования
    if gender_or_number != 'Plur' and noun['feats']['Gender'] != gender_or_number: #в другом роде слово точно не согласуется
        return False
    if gender_or_number == 'Plur': #теперь нам нужно только число
        number = 'Plur'
    else:
        number = 'Sing'
    possible_tags = get_possible_tags_noun(udpipe_noun, homonimic_dict)
    if (case, number) in possible_tags:
        return True
    else:
        return False

In [ ]:
##Проверяет, может ли существительное быть формой указанного падежа
def check_government_homonimy(noun, case):
    possible_tags = get_possible_tags_noun(udpipe_noun, homonimic_dict)
    possible_cases = [tag[0] for tag in possible_tags]
    if case in possible_cases:
        return True
    else:
        return False

In [ ]:
##Ищет среди потомков существительные, помеченные юдипайпом как иминительный или винительный (второе - только для неодушевленных)
def subject_candidates_in_children(sentence, predicative_head, candidates_list):
    children_nouns = sentense.filter(upostag='noun', head=predicative_head['id'])
    for noun in children_nouns:
        if (noun['feats']['Сase'] == ['Nom']) or (noun['feats']['Сase'] == ['Acc'] and noun['feats']['Animacy'] = 'Inan'):
            candidates_list.append(noun)
    return candidates_list

In [ ]:
#ищет кандидаты в подлежащее среди потомков и потомков родителей, останавливается на первой личной форме глагола
def subject_candidates(sentence, predicative_head):
    candidates = []
    candidates += subject_candidates_in_children(predicative_head)
    while predicative_head['upostag'] != 'VERB': #вторая встреченная личная форма может быть в другой части предложения
        predicative_head = sentence.filter(id=predicative_head['parent'])
        if predicative_head['upostag'] == 'CONJ': ##в случае однородности берём не союз, а его вершину
            predicative_head = sentence.filter(id=predicative_head['parent'])
        if predicative_head['upostag'] in ['VERB', 'INF', 'ADJ']: 
            candidates += subject_candidates_in_children(predicative_head)
        else:
            break
    return candidates

In [ ]:
## принимает предложение, возвращает список потенциальных ошибок в прилагательных
def check_adjectives(sentence):
    mistakes = []
    adjectives = get_adjectives_dict(sentence)
    for adjective in adjectives['full']:
        head = sentence.filter(id=adjective['head'])
        if head['upostag'] == 'NOUN': ## 
            if not (adjective['feats']['case']== head['feats']['Case'] and get_number_or_gender(adjective) == get_number_or_gender(noun)):
                if not check_agreement_homonimy(noun, adjective['feats']['case'], get_number_or_gender(adjective)):
                    mistakes.append(adjective)
        else:
            if adjective['feats'] not in ['Nom', 'Ins']:
                mistakes.append(adjective)
            else:
                adjectives['predicative'].append(adjective) ##проверка согласования по роду/числу - как для кратких
    for adjective in adjectives['predicative']:
        head = adjective['head']
        subject_candidates = subject_candidates(sentence, head)
        adj_agreement_feature = get_number_or_gender(adjective)
        if not any([get_number_or_gender(subject_candidate) for subject_candidate in subject_candidates]):
            mistakes.append(adjective)
    return mustakes